In [ ]:
import pandas as pd
df1=pd.read_csv('oneminuteGround2015FULL.csv')
df2=pd.read_csv('oneminuteGround2016FULL.csv')
df3=pd.read_csv('oneminuteGround2017FULL.csv')
df4=pd.read_csv('oneminuteGround2018FULL.csv')

In [ ]:
vd=pd.concat([df1,df2,df3,df4],axis=0)

In [ ]:
df=vd

In [ ]:
f=rf.drop([ 'InvVa_Avg', 'InvVb_Avg', 'InvVc_Avg', 'InvIa_Avg',
       'InvIb_Avg', 'InvIc_Avg', 'InvFreq_Avg', 'InvPAC_kW_Avg',
       'InvEtot_kWh_Max', 'InvVPVin_Avg', 'InvPDC_kW_Avg', 'InvOpStatus_Avg',
       'InvMainFault_Max', 'InvDriveFault_Max', 'InvVoltageFault_Max',
       'InvGridFault_Max', 'InvTempFault_Max', 'InvSystemFault_Max',
       'InvSystemWarn_Max', 'InvPVMStatus_Max', 'SEWSAmbientTemp_C_Avg',
       'SEWSModuleTemp_C_Avg', 'SEWSPOAIrrad_Wm2_Avg', 'PwrMtrIa_Avg',
       'PwrMtrIb_Avg', 'PwrMtrIc_Avg', 'PwrMtrFreq_Avg', 'PwrMtrFreqMin_Min',
       'PwrMtrFreqMax_Max', 'PwrMtrPhaseRev_Avg', 'PwrMtrVa_Avg',
       'PwrMtrVb_Avg', 'PwrMtrVc_Avg', 'PwrMtrP_kW_Avg', 'PwrMtrP_kVAR_Avg',
       'PwrMtrP_kVA_Avg', 'PwrMtrEdel_kWh_Max', 'PwrMtrErec_kWh_Max',
       'PwrMtrEdel_kVARh_Max', 'PwrMtrErec_kVARh_Max', 'PwrMtrPF_Avg',
       'PwrMtrVaTHDMax_Max', 'PwrMtrVbTHDMax_Max', 'PwrMtrVcTHDMax_Max',
       'ShuntCurrent_A_Avg_1', 'ShuntCurrent_A_Avg_2', 'ShuntCurrent_A_Avg_3',
       'ShuntCurrent_A_Avg_4', 'ShuntCurrent_A_Avg_5', 'ShuntCurrent_A_Avg_6',
       'ShuntCurrent_A_Avg_7'],axis=1)

In [ ]:
f=f.drop([ 'WindDirStdDev_deg',
       'WindSpeed_ms_Max', 'WindRef_V_Min', 'WindHeatStateID_Avg',
       'WindValid_Avg', 'Battery_V_Min', 'Battery_A_Avg', 'Load_A_Avg',
       'ChgState_Min', 'ChgSource_Min', 'CkBatt_Max', 'Qloss_Ah_Max'],axis=1)

In [ ]:
df.replace({'-05:00': ' '}, regex=True)

Feature Engineering

In [ ]:
date_time= pd.to_datetime(df['TIMESTAMP'],format='%Y-%m-%d %H:%M')

In [ ]:
import datetime
timestamp_s = date_time.map(datetime.datetime.timestamp)

In [ ]:
import numpy as np

In [ ]:
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))


In [ ]:
threshold = 0.05 # Anything that occurs less than this will be removed.

value_counts = df['Day sin'].value_counts() # Specific column 
to_remove = value_counts[ abs(value_counts) <= threshold].index
df['Day sin'].replace(to_remove, 0, inplace=True)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.plot(np.array(df['Day sin'])[:1440],label='Day sin')
plt.plot(np.array(df['Day cos'])[:1440],label='Day cos')
plt.xlabel('Time [min]')
plt.legend(loc='lower right')
plt.title('Time of day signal')

In [ ]:
plt.plot(np.array(df['Year sin'])[:525600],label='Year sin')
plt.plot(np.array(df['Year cos'])[:525600],label='Year cos')
plt.xlabel('Time [min]')
plt.legend(loc='lower left')
plt.title('Time of year signal')

In [ ]:
df.pop('TIMESTAMP')

In [ ]:
wv = df.pop('WindSpeedAve_ms')
wd_rad = df.pop('WindDirAve_deg')*np.pi / 180
df['Wx'] = wv*np.cos(wd_rad)
df['Wy'] = wv*np.sin(wd_rad)

Averaging of columns

In [ ]:
new_col_RTD = ((df['RTD_C_Avg_1']+df['RTD_C_Avg_2']+df['RTD_C_Avg_3']+df['RTD_C_Avg_4']+df['RTD_C_Avg_5']+df['RTD_C_Avg_6']+df['RTD_C_Avg_7']+df['RTD_C_Avg_8']+df['RTD_C_Avg_9']+df['RTD_C_Avg_10'])/10).tolist()
df.insert(loc=len(df.columns), column='RTD_avg', value=new_col_RTD)

In [ ]:
df['Angle_degrees']=20

In [ ]:
new_col_P = ((df['Pyra1_Wm2_Avg']+df['Pyra2_Wm2_Avg'])/2).tolist()
df.insert(loc=len(df.columns), column='Pyra_Wm2_Avg', value=new_col_P)

In [ ]:
#new_col = (df.iloc[:,20]+df.iloc[:,22]+df.iloc[:,21]+df.iloc[:,23]).tolist()
new_col = (df['ShuntPDC_kW_Avg_1']+df['ShuntPDC_kW_Avg_2']+df['ShuntPDC_kW_Avg_3']+df['ShuntPDC_kW_Avg_4']+df['ShuntPDC_kW_Avg_5']+df['ShuntPDC_kW_Avg_6']+df['ShuntPDC_kW_Avg_7']).tolist()
df.insert(loc=len(df.columns), column='power generated', value=new_col)

In [ ]:
c=df['ShuntEtot_kWh_Max_1']+ df['ShuntEtot_kWh_Max_2']+df['ShuntEtot_kWh_Max_3']+df['ShuntEtot_kWh_Max_4']+df['ShuntEtot_kWh_Max_5']+df['ShuntEtot_kWh_Max_6']+df['ShuntEtot_kWh_Max_7']
df.insert(loc=len(df.columns), column='energy generated', value=c)

In [ ]:
df.head()

Remove individual columns of power generation

In [ ]:
df.pop('ShuntEtot_kWh_Max_1')
df.pop('ShuntEtot_kWh_Max_2')
df.pop('ShuntEtot_kWh_Max_3')
df.pop('ShuntEtot_kWh_Max_4')
df.pop('ShuntEtot_kWh_Max_5')
df.pop('ShuntEtot_kWh_Max_6')
df.pop('ShuntEtot_kWh_Max_7')

df.pop('ShuntPDC_kW_Avg_1')
df.pop('ShuntPDC_kW_Avg_2')
df.pop('ShuntPDC_kW_Avg_3')
df.pop('ShuntPDC_kW_Avg_4')
df.pop('ShuntPDC_kW_Avg_5')
df.pop('ShuntPDC_kW_Avg_6')
df.pop('ShuntPDC_kW_Avg_7')
df.pop('RTD_C_Avg_1')
df.pop('RTD_C_Avg_2')
df.pop('RTD_C_Avg_3')
df.pop('RTD_C_Avg_4')
df.pop('RTD_C_Avg_5')
df.pop('RTD_C_Avg_6')
df.pop('RTD_C_Avg_7')
df.pop('RTD_C_Avg_8')
df.pop('RTD_C_Avg_9')
df.pop('RTD_C_Avg_10')
df.pop('Pyra1_Wm2_Avg')
df.pop('Pyra2_Wm2_Avg')



Impute data

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_filled = imputer.fit_transform(df)
#df=pd.DataFrame(imp.fit_transform(df.iloc[:,:].values.reshape(-1,1)))

Save to csv


In [ ]:
df_filled.to_csv('groundfinal.csv',index=False)